In [1]:
import sys
sys.path.append("/Users/nikhil/workspace/models/research/slim/")
sys.path.append("/Users/nikhil/workspace/models/research/slim/nets/")
import tensorflow as tf
import numpy as np
import resnet_v2
from six.moves import cPickle as pickle
import sys
import math
import time

/Users/nikhil/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [16]:
def load_pickled_dataset(pickle_file):
    print("Loading datasets...")
    with open(pickle_file, 'rb') as f:
        save = pickle.load(f, encoding='latin1')
        train_X = save['train_data']
        train_Y = save['train_labels']
        val_X = save['val_data']
        val_Y = save['val_labels']
        test_X = save['test_data']
        test_Y = save['test_labels']
        del save  # hint to help gc free up memory
    print('Training set', train_X.shape, train_Y.shape)
    print('Validation set',val_X.shape,val_Y.shape)
    print('Test set',test_X.shape,test_Y.shape)
    return (train_X, train_Y, val_X, val_Y, test_X, test_Y)


In [9]:
from tensorflow.python.ops import control_flow_ops
slim = tf.contrib.slim

def weight_variable(shape, name):
    return tf.get_variable(name + "/weights", shape=shape,
                      initializer=tf.contrib.layers.xavier_initializer())

def bias_variable(shape, name):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial, name=name + "/biases")

def add_batch_norm_dependency(total_loss):
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    if update_ops:
        updates = tf.tuple(update_ops)
        total_loss = control_flow_ops.with_dependencies(updates, total_loss)
    return total_loss

In [11]:
class ResNet50(object):
    def __init__(self, inputs, labels, learning_rate, is_training, n_classes, reuse, name=""):
        with tf.variable_scope(name):
            with slim.arg_scope(resnet_v2.resnet_arg_scope()):
                if reuse == True:
                    tf.get_variable_scope().reuse_variables()
                net, _ = resnet_v2.resnet_v2_50(inputs, is_training=is_training, spatial_squeeze=True)
                self.net_saver = tf.train.Saver()
                self.net = tf.reshape(net, [-1, 2048])
                W = weight_variable([2048, n_classes], "output")
                b = bias_variable([n_classes], "output")
                self.raw_output = tf.matmul(self.net, W) + b
                self.output = tf.sigmoid(self.raw_output)
                self.accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(self.output, 1), tf.argmax(labels, 1)), tf.float32))
                self.loss = tf.nn.sigmoid_cross_entropy_with_logits(
                logits=self.raw_output, labels=labels)
                self.loss = tf.reduce_mean(self.loss)
                self.loss = add_batch_norm_dependency(self.loss)
                if reuse == None:
                    self.train_step = tf.train.AdamOptimizer(learning_rate).minimize(self.loss)
                self.saver = tf.train.Saver()

In [12]:
tf.reset_default_graph()
learning_rate = 1e-4
train_images = tf.placeholder(tf.float32, [None, 224, 224, 3])
train_labels = tf.placeholder(tf.float32, [None, 10])
valid_images = tf.placeholder(tf.float32, [None, 224, 224, 3])
valid_labels = tf.placeholder(tf.float32, [None, 10])

In [13]:
train_net = ResNet50(train_images, train_labels, learning_rate, True, 10, None)
valid_net = ResNet50(train_images, train_labels, learning_rate, False, 10, True)

In [14]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [14]:
train_net.net_saver.restore(sess, "/Users/nikhil/Downloads/resnet_v2_50.ckpt")

In [21]:
PICKLE_FILE = "/Users/nikhil/Desktop/Project/medical_data/medical_k_hot_data.pickle"
(train_X, train_Y, val_X, val_Y, test_X, test_Y) = load_pickled_dataset(PICKLE_FILE)

Loading datasets...
Training set (75, 224, 224, 3) (75, 15)
Validation set (15, 224, 224, 3) (15, 15)
Test set (15, 224, 224, 3) (15, 15)


In [22]:
print(train_X.shape)
train_array = sess.run(train_net.net, feed_dict = {train_images : train_X} )
print (train_array.shape)
np.save("/Users/nikhil/Desktop/Project/sample_train_array.npy", train_array)
np.load("/Users/nikhil/Desktop/Project/sample_train_array.npy").shape

(75, 224, 224, 3)
(75, 2048)


(75, 2048)

In [41]:
import mobilenet_v1
class MobileNet25(object):
    def __init__(self, inputs, labels, learning_rate, is_training, n_classes, reuse, name=""):
        with tf.variable_scope(name):
            with slim.arg_scope(mobilenet_v1.mobilenet_v1_arg_scope()):
                if reuse == True:
                    tf.get_variable_scope().reuse_variables()
                net, _ = mobilenet_v1.mobilenet_v1_025(inputs, 1001, is_training=is_training, spatial_squeeze=True)
                print(net.get_shape())
                self.net_saver = tf.train.Saver()
                self.net = tf.reshape(net, [-1, 1001])
                W = weight_variable([1001, n_classes], "output")
                b = bias_variable([n_classes], "output")
                self.raw_output = tf.matmul(self.net, W) + b
                self.output = tf.sigmoid(self.raw_output)
                self.accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(self.output, 1), tf.argmax(labels, 1)), tf.float32))
                self.loss = tf.nn.sigmoid_cross_entropy_with_logits(
                logits=self.raw_output, labels=labels)
                self.loss = tf.reduce_mean(self.loss)
                self.loss = add_batch_norm_dependency(self.loss)
                if reuse == None:
                    self.train_step = tf.train.AdamOptimizer(learning_rate).minimize(self.loss)
                self.saver = tf.train.Saver()

In [42]:
tf.reset_default_graph()
learning_rate = 1e-4
train_images = tf.placeholder(tf.float32, [None, 224, 224, 3])
train_labels = tf.placeholder(tf.float32, [None, 15])
valid_images = tf.placeholder(tf.float32, [None, 224, 224, 3])
valid_labels = tf.placeholder(tf.float32, [None, 15])

In [43]:
train_net = MobileNet25(train_images, train_labels, learning_rate, True, 15, None)
valid_net = MobileNet25(train_images, train_labels, learning_rate, False, 15, True)

(?, 1001)
(?, 1001)


In [44]:
train_net.net.get_shape()
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [45]:
train_net.net_saver.restore(sess, "/Users/nikhil/Downloads/Temporary Downloads/mobilenet_v1_0.25_224_2017_06_14/mobilenet_v1_0.25_224.ckpt")

INFO:tensorflow:Restoring parameters from /Users/nikhil/Downloads/Temporary Downloads/mobilenet_v1_0.25_224_2017_06_14/mobilenet_v1_0.25_224.ckpt


In [46]:
PICKLE_FILE = "/Users/nikhil/Desktop/Project/medical_data/medical_k_hot_data.pickle"
(train_X, train_Y, val_X, val_Y, test_X, test_Y) = load_pickled_dataset(PICKLE_FILE)
print(train_X.shape)
train_array =  sess.run(train_net.net, feed_dict = {train_images :train_X} )
print (train_array.shape)
np.save("/Users/nikhil/Desktop/Project/mobilenet_k_hot_array.npy", train_array)
np.load("/Users/nikhil/Desktop/Project/mobilenet_k_hot_array.npy").shape

Loading datasets...
Training set (150, 224, 224, 3) (150, 15)
Validation set (150, 224, 224, 3) (150, 15)
Test set (150, 224, 224, 3) (150, 15)
(150, 224, 224, 3)
(150, 1001)


(150, 1001)

In [47]:
%timeit sess.run(train_net.train_step, feed_dict = {train_images : train_X, train_labels : train_Y})

KeyboardInterrupt: 

In [94]:
sess.run(valid_net.accuracy, feed_dict = {valid_images: valid_X, valid_labels: valid_Y })

TensorShape([Dimension(None), Dimension(1000)])